## Uncapacitated Facility Location problem

In [ ]:
using Random, Statistics, LinearAlgebra

Random.seed!(79883274598)

nI = 100  # number of facilities
nJ = 100  # number of customers

# generate random problem instance
f = 0.5*ones(nI)
c = rand(nI,nJ)

using JuMP, Gurobi
m = Model(Gurobi.Optimizer)
#set_silent(m)

@variable(m, z[1:nI], Bin)         # binary constraint
# @variable(m, 0 <= z[1:nI] <= 1)    # LP relaxation

@variable(m, y[1:nI,1:nJ], Bin)     # binary constraint
#@variable(m, y[1:nI,1:nJ] >= 0)   # LP relaxation

@constraint(m, cc[j = 1:nJ], sum( y[i,j] for i=1:nI ) == 1)

# choose one of these two constraints (they are equivalent)
# @constraint(m, cr[i=1:nI], sum( y[i,j] for j=1:nJ ) <= nJ*z[i])
@constraint(m, cr[i=1:nI,j=1:nJ], y[i,j] <= z[i])

@objective(m, Min, dot(f,z) + dot(c,y))

@time(optimize!(m))

println(value.(z))
zopt=value.(z)
println("total number of facilities is ",sum(zopt))

In [ ]:
using JuMP, Gurobi, HiGHS

nvals = [5,10,15,20,25,30,35,40,45,50]
tt_highs = zeros(length(nvals),50)
tt_gurobi = zeros(length(nvals),50)
;

In [ ]:
t = tt_highs
for count = 1:10
    println(count)
    for (k,n) in enumerate(nvals)
        nI = n  # number of facilities
        nJ = n  # number of customers

        # generate random problem instance
        Random.seed!(count)
        f = 0.5*ones(nI)
        c = rand(nI,nJ)

        m = Model(HiGHS.Optimizer)
        set_silent(m)

        @variable(m, x[1:nI], Bin)         # binary constraint
        @variable(m, y[1:nI,1:nJ] >= 0)    # LP relaxation
        @constraint(m, cc[j = 1:nJ], sum( y[i,j] for i=1:nI ) == 1)
        @constraint(m, cr[i=1:nI,j=1:nJ], y[i,j] <= x[i])
        @objective(m, Min, dot(f,x) + dot(vec(c),vec(y)))
        t[k,count] = @timed(optimize!(m))[2]
    end
end

In [ ]:
t = tt_gurobi
for count = 1:10
    println(count)
    for (k,n) in enumerate(nvals)
        nI = n  # number of facilities
        nJ = n  # number of customers

        # generate random problem instance
        Random.seed!(count)
        f = 0.5*ones(nI)
        c = rand(nI,nJ)

        m = Model(Gurobi.Optimizer)
        set_silent(m)

        @variable(m, x[1:nI], Bin)         # binary constraint
        @variable(m, y[1:nI,1:nJ] >= 0)    # LP relaxation
        @constraint(m, cc[j = 1:nJ], sum( y[i,j] for i=1:nI ) == 1)
        @constraint(m, cr[i=1:nI,j=1:nJ], y[i,j] <= x[i])
#         @objective(m, Min, dot(f,x) + vecdot(c,y))
         @objective(m, Min, dot(f,x) + dot(vec(c),vec(y)))
        t[k,count] = @timed(optimize!(m))[2]
    end
end

In [ ]:
# println(tt_gurobi)
# for (k,n) in enumerate(nvals)
#     println(k, ":", n)
# end 

tt_highs_mean = []
for (k,n) in enumerate(nvals)
    x = tt_highs[k,:]
    x = [j for j in x if j > 0]
    append!(tt_highs_mean, mean(x))
end

tt_gurobi_mean = []
for (k,n) in enumerate(nvals)
    x = tt_gurobi[k,:]
    x = [j for j in x if j > 0]
    append!(tt_gurobi_mean, mean(x))
end
println(tt_gurobi_mean)
println(tt_highs_mean)


#tt_gurobi_mean = mean(tt_gurobi,2)

# tt_highs_mean = mean(tt_highs,2)




In [ ]:
using PyPlot

figure(figsize=(10,4))
semilogy(nvals,tt_highs_mean,label="HiGHS")
semilogy(nvals,tt_gurobi_mean,label="Gurobi")
legend(loc="best")
xlabel("number of facilities and customers")
ylabel("time to solve (seconds)")
title("comparison of different solvers")
tight_layout()

display(gcf())

#savefig("solver_comparison.pdf")
;